### Designing a finite state automaton in Python

CS 236 <br>
Fall 2023

Michael A. Goodrich <br>
Brigham Young University <br>
Fall 2023
***

The textbook defines a Finite State Automaton (FSA) as a tuple (see Def 3 in section 13.3.3). Think of a _tuple_ as an ordered pair with more than two elements. The tuple for an FSA is $(S,I,s_0,F,f)$ where 
* A finite set of states $S$
* A set of input characters $I$
* A start state $s_0$
* A set of final or accept states $F$
* A transition function $f:S\times I \rightarrow S$
<br>


We often draw FSAs by representing each state $s\in S$ with a circle, representing all states in $F$ with a double circle, representing the start state by a circle with an arrow pointing toward it, and representing transition functions as arrows connecting one state to another. The label of the arrow connecting one state to another is the input $i\in I$ that causes the FSA to _transition_from one state to another. 

Consider the following FSA

![colon-dash FSA](colon-dash-fsa.png)

The elements of the FSA tuple above are
* The finite set of states is $S=\{s_0,s_1,s_2,s_{\rm err}\}$
* The finite set of input characters is $I=\{{\rm any\  keyboard\  character}\}$
* The start state is $s_0$
* The set of final or accept states is $F=\{s_2\}$
* The transition function $f:S\times I \rightarrow S$ are all the states connected by the labeled arrows:

| present state | input             | next state |
| :-:           | :-:               | :-:        |
| $s_0$         | ':'               | $s_1$      |
| $s_1$         | '-'               | $s_2$      |
| $s_2$         | anything          | $s_2$      |
| $s_0$         | anything but ':'  | $s_{err}$  |
| $s_1$         | anything but '-'  | $s_{err}$  |
| $s_{err}$     | anything          | $s_{err}$  |

Notice how we used some shortcuts, like writing "anything" on a single transition rather than creating a transition for every possible input from $s_2$ to $s_2$ or from $s_{err}$ to $s_{err}$. We did something similar for that transition from $s_0$ or $s_1$ to $s_{err}$, labeling a transition in a way that says that any input except for ":" or "-", respectively, transitioned to $s_{err}$. 

***


We can create a class that implements this FSA. We'll define two types of things in the constructor. First, we'll need to define everything required in the definition of a FSA: set of states, set if input characters, start state, set of accept states, and transition function. Second, we'll define some variables that we'll use to manage the input string or that will be used in a subsequent tool.

***



Let's take a look at just a portion of the class with a very simple constructor and one user-defined function that doesn't do anything. We'll do this so that we can contrast the class definition without the style guide to the class definition that uses the style guide. 

In [ ]:
class ColonDashFSA:
    def __init__(self):
        """ Class constructor """
        ###############################################################
        # Define the five elements of the FSA
        ###############################################################
        # Set of states: Each state s0, s1, s2, s_err will be represented by its own function
        # Set of inputs: I is the set of alphanumeric characters, checked by isalnum()
        self.start_state = self.s0  # Initialize the starting state to the correct function
        self.accept_states = set(); 
        # Transition function: Each transition will be defined in the state functions
        
        ###############################################################
        # Define four variables that are used within the FSA, some
        # to make the FSA run and some that help us understand how 
        # the FSA works
        ###############################################################
        self.input_string = ""      # Default empty input
        self.fsa_name = "colon-dash state machine"
        
    def s0(self) -> function:
        # Not implemented ... just for illustration
        next_state = None
        return next_state

Let's now look at the same code that uses the style guide. (You can find the style guide under Course Content on Learning Suite. It's also saved as a file in this project so that you can reference it from within this tutorial.) 


In [ ]:
# This import gives us the type hint "function". It is renamed from "Callable"
from typing import Callable as function

class ColonDashFSA:
    def __init__(self) -> None:
        """ Class constructor """
        ###############################################################
        # Define the five elements of the FSA
        ###############################################################
        # Set of states: Each state s0, s1, s2, s_err will be represented by its own function
        # Set of inputs: I is the set of alphanumeric characters, checked by isalnum()
        self.start_state: function = self.s0  # Initialize the starting state to the correct function
        self.accept_states: set[function] = set(); 
        # Transition function: Each transition will be defined in the state functions
        
        ###############################################################
        # Define four variables that are used within the FSA, some
        # to make the FSA run and some that help us understand how 
        # the FSA works
        ###############################################################
        self.input_string: str = ""      # Default empty input
        self.fsa_name: str = "colon-dash state machine"
            
    #########################################################
    # Define each state as a function                       #
    # Within each function, define the transition function. #
    # Each transition function reads the current input and  #
    # choose the next state based on the input              #
    #########################################################
    def s0(self) -> function:
        next_state: function = None
        return next_state



The first change is that we'll import a portion of the _typing_ package using

    from typing import Callable as function

Each object in python has a type, similar to what you saw in the data structures class for C++. But python has a lot of ways that you can mess with (and mess up) the type, so you are required to specify the types of each function and each variable you define. Tye _typing_ package is a collection of tools that can be used to be clear about the types of your functions and variables.

Quoting from ChatGPT from the prompt "What is a callable in python?" In python, a _Callable_ is an object that can be called as a function. There are different types of callables in python, but we'll only be using the built in functions like print and functions defined in the classes. The import statement says that we're going to use _Callable_ types but since we are are probably more familiar with the term _function_ we'll just call all _Callables_ by the name _function_.

***

The second change between the "no style" class definition and the class definition that follows the style guide is

    def __init__(self) -> None:

Contrast this with what was in the "no style" class

    def __init__(self):

The "->" notation indicates that we are going to make a kind of contract with ourselves (or other programmers) about what this function returns. In this case, the constructor doesn't return anything, so we say "-> None", which we can read as _this function doesn't return anything_.

Now consider how we define the S0 function

    def s0(self) -> function:

Contrast this with what was in the "no style" class

    def s0(self):

We read the "-> function" as _this function returns another function_. We'll discuss this more later.

***


The third change is in how we define variables. In the "no style" class, we used

        self.start_state = self.s0  # Initialize the starting state to the correct function
        self.accept_states = set();
        self.input_string = ""      # Default empty input
        self.fsa_name: str = "colon-dash state machine"

and in the class that follows the style guide we used

        self.start_state: function = self.s0  # Initialize the starting state to the correct function
        self.accept_states: set[function] = set(); 
        self.input_string: str = ""      # Default empty input
        self.fsa_name: str = "colon-dash state machine"

Notice how after the name of the variable we put a colon, followed by the variable type, followed by an equal sign, and then followed by the value assigned to the variable. We read the four lines of code above as 

 * The type of the _self.start_state_ variable is function, and we set the value of the variable to _self.s0_.
 * The type of the _self.accept_states_ variable is a set whose member elements are functions, and we set the value of the variable to the empty set.
 * The types of both _self.input_string_ and _self.fsa_name_ are both strings, one initialized to the empty string and the other initialized to the string "colon-dash state machine".

 ***


We can now explore code that actually works.

After the constructor, a function is defined for each state. (Note that this is not the most efficient implementation of a state machine, but it's one of the easiest to understand. If you take CS 340, you'll learn about _design patterns_ that frequently appear in software engineering. One of these design patterns is the _state machine pattern_.) Each function has the same pattern: get the current input and then transitions to the next state determined by the current input. (Ignore the history_message stuff. We'll use it later)

In [4]:
# This import gives us the type hint "function". It is renamed from "Callable"
from typing import Callable as function

class ColonDashFSA:
    def __init__(self) -> None:
        """ Class constructor """
        ###############################################################
        # Define the five elements of the FSA
        ###############################################################
        # Set of states: Each state s0, s1, s2, s_err will be represented by its own function
        # Set of inputs: I is the set of alphanumeric characters, checked by isalnum()
        self.start_state: function = self.s0  # Initialize the starting state to the correct function
        self.accept_states: set[function] = set(); 
        self.accept_states.add(self.s2)  
        # Transition function: Each transition will be defined in the state functions
        
        ###############################################################
        # Define four variables that are used within the FSA, some
        # to make the FSA run and some that help us understand how 
        # the FSA works
        ###############################################################
        self.input_string: str = ""      # Default empty input
        self.fsa_name: str = "colon-dash state machine"
        self.num_chars_read: int = 0
        self.history: list[str] = []
            
    #########################################################
    # Define each state as a function                       #
    # Within each function, define the transition function. #
    # Each transition function reads the current input and  #
    # choose the next state based on the input              #
    #########################################################
    def s0(self) -> function:
        current_input = self.__get_current_input()
        next_state: function = None
        history_message: str = ""
        if current_input == ':': 
            next_state = self.s1
            history_message = self.get_history_string("s0",current_input,"s1")
        else: 
            next_state = self.s_err
            history_message = self.get_history_string("s0",current_input,"s_err")
        self.history.append(history_message)
        return next_state

    def s1(self) -> function:
        current_input = self.__get_current_input()
        next_state: function = None
        history_message: str = ""
        if current_input == '-': 
            next_state = self.s2
            history_message = self.get_history_string("s1",current_input,"s2")
        else: 
            next_state = self.s_err
            history_message = self.get_history_string("s1",current_input,"s_err")
        self.history.append(history_message)
        return next_state

    def s2(self) -> function:
        current_input: str = self.__get_current_input()
        next_state: function = self.s2 # loop in state s2
        history_message: str = self.get_history_string("s2",current_input,"s2")
        self.history.append(history_message)
        return next_state

    def s_err(self) -> function:
        current_input: str = self.__get_current_input()
        next_state: function = self.s_err # loop in state s_err
        history_message: str = self.get_history_string("s_err",current_input,"s_err")
        self.history.append(history_message)
        return next_state
    
    ############################
    # Manager functions
    ############################
    def run(self, input_string: str) -> bool:
        ###############################################################
        # This function will be called to make the FSA execute
        # It records the input string,
        # sets the current state to the start state
        # and then calls a state function that returns the next state
        # Each state function accesses the input_string
        ###############################################################
            # Remember input_string
        self.input_string = input_string
            # Set current state to start state
        current_state: function = self.start_state
            # Call current state, which starts the FSA
        while self.num_chars_read < len(self.input_string):
            current_state = current_state()
            # Check whether the FSA ended in an accept state
        outcome: bool = False 
        if current_state in self.accept_states: outcome = True # Accept if the FSA ended in an accept state
        return outcome

    def reset(self):
        self.num_chars_read = 0
        self.history = []
    
    ############################
    # Public Getters and Setters
    ############################
    def get_name(self) -> str: return self.fsa_name
    def get_history(self) -> str: 
        output_string: str = ""
        for message in self.history:
            output_string = output_string + message + "\n"
        return output_string

    def get_history_string(self, current_state: function, input: str, next_state: function):
        history_message: str = "From state " + current_state + " read input " + \
            input + " and transitioned to state " + next_state
        return history_message
    
    ############################
    # Private Helper functions
    ############################
    def __get_current_input(self) -> str:  # The double underscore makes the function private
        current_input: str = self.input_string[self.num_chars_read]
        self.num_chars_read += 1
        return current_input
    

Let's test whether the FSA we defined actually works by running it on an input

In [5]:
my_fsa: ColonDashFSA = ColonDashFSA()
input_string: str = ":-"
accept_status: bool = my_fsa.run(input_string)
if accept_status: print("The ", my_fsa.get_name(), "FSA accepted the input string '", input_string, "'")
else: print("The ", my_fsa.get_name(), "FSA did not accept the input string '", input_string, "'")


The  colon-dash state machine FSA accepted the input string ' :- '


Yay! it worked. 

It's useful to print out the _trace_ of the FSA. The _trace_ is defined as the history of each state that was visited, the input character read from that state, and the state to which the FSA transitioned given that input.

***


In [6]:
my_fsa.reset()  # This function resets the FSA to run on a new input.
input_string: str = ":-"
accept_status: bool = my_fsa.run(input_string)
if accept_status: print("The ", my_fsa.get_name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_fsa.get_name(), "FSA did not accept the input string '",input_string,"'")
print("Here is the trace on input '",input_string,"'")
print(my_fsa.get_history())

The  colon-dash state machine FSA accepted the input string ' :- '
Here is the trace on input ' :- '
From state s0 read input : and transitioned to state s1
From state s1 read input - and transitioned to state s2



Just as expected, the FSA started in the start state, s0, read the colon and transitioned to state s1, and then read the dash and transitioned to state s2. The trace shows the sequence of states visited and inputs read.

***

Let's try an input string that should not be accepted by the FSA.

In [7]:
my_fsa.reset()
input_string: str = ":"
accept_status: bool = my_fsa.run(input_string)
if accept_status: print("The ", my_fsa.get_name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_fsa.get_name(), "FSA did not accept the input string '",input_string,"'")
print("Here is the trace on input '",input_string,"'")
print(my_fsa.get_history())

The  colon-dash state machine FSA did not accept the input string ' : '
Here is the trace on input ' : '
From state s0 read input : and transitioned to state s1



***
Let's try an input that is a little longer.

In [9]:
my_fsa.reset()
input_string: str = ":::-"
accept_status: bool = my_fsa.run(input_string)
if accept_status: print("The ", my_fsa.get_name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_fsa.get_name(), "FSA did not accept the input string '",input_string,"'")
print("Trace the following history using the picture of the FSA above:")
print(my_fsa.get_history())

The  colon-dash state machine FSA did not accept the input string ' :::- '
Trace the following history using the picture of the FSA above:
From state s0 read input : and transitioned to state s1
From state s1 read input : and transitioned to state s_err
From state s_err read input : and transitioned to state s_err
From state s_err read input - and transitioned to state s_err



***
Let's try a long input that should be accepted.

In [28]:
my_fsa.reset()
input_string: str = ":-::- testing"
accept_status: bool = my_fsa.run(input_string)
if accept_status: print("The ", my_fsa.get_name(), "FSA accepted the input string '",input_string,"'")
else: print("The ", my_fsa.get_name(), "FSA did not accept the input string '",input_string,"'")
print("Trace the following history using the picture of the FSA above:")
print(my_fsa.get_history())

The  colon-dash state machine FSA accepted the input string ' :-::- testing '
Trace the following history using the picture of the FSA above:
From state S0 read input : and transitioned to state S1
From state S1 read input - and transitioned to state S2
From state S2 read input : and transitioned to state S2
From state S2 read input : and transitioned to state S2
From state S2 read input - and transitioned to state S2
From state S2 read input   and transitioned to state S2
From state S2 read input t and transitioned to state S2
From state S2 read input e and transitioned to state S2
From state S2 read input s and transitioned to state S2
From state S2 read input t and transitioned to state S2
From state S2 read input i and transitioned to state S2
From state S2 read input n and transitioned to state S2
From state S2 read input g and transitioned to state S2

